In [1]:
import requests
import numpy as np
import pandas as pd
import bs4 as bs
import lxml.html as lh
import folium

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
def scrape_table_lxml(XPATH,cols):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    table_content = doc.xpath(XPATH)
    for table in table_content:
        headers = [th.text_content().strip() for th in table.xpath('//th')]
        headers = headers[0:3]
        data    = [[td.text_content().strip() for td in tr.xpath('td')] 
                   for tr in table.xpath('//tbody/tr')]
        data    = [row for row in data if len(row) == cols]
        raw_df = pd.DataFrame(data,columns=headers)
        return raw_df

In [4]:
XPATH = "/html/body/div[3]/div[3]/div[4]/div/table[1]"

In [5]:
mydata = scrape_table_lxml(XPATH,3)

In [6]:
TorontoPostalCodes=mydata[~mydata['Borough'].isin(['Not assigned'])]
TorontoPostalCodes=TorontoPostalCodes.sort_values(by=['Postcode','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)
TorontoPostalCodes.loc[TorontoPostalCodes['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = TorontoPostalCodes['Borough']
check_unassigned_post_state_sample = TorontoPostalCodes.loc[TorontoPostalCodes['Borough'] == 'Queen\'s Park']
TorontoPostalCodes = TorontoPostalCodes.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [7]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'
geocsv_data = pd.read_csv(toronto_geocsv)

In [8]:
toronto_neighborhoods = TorontoPostalCodes.join(geocsv_data)
toronto_neighborhoods.drop('Postal Code', axis=1, inplace=True)
toronto_neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:


# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto



In [10]:
toronto_neighborhoods

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [11]:
toronto_data = toronto_neighborhoods[toronto_neighborhoods['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [12]:
west_toronto_data = toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'West Toronto'].reset_index(drop=True)

In [13]:
west_toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
3,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
4,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
5,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450


In [14]:
CLIENT_ID = 'BNWKW22EZCDC2PBW13BYIP0H2H3VVCBR1LKCXMPF4METKIXP'
CLIENT_SECRET = 'YPPP00QC5GBB0LADRW1SXGB22OAVRWOZDIVSJ3OJGQFRT35C' # your Foursquare Secret
VERSION = '20190217'

LIMIT = 100
radius = 300

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
Riverdale, The Danforth West
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",57,57,57,57,57,57
"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",4,4,4,4,4,4
Berczy Park,7,7,7,7,7,7
"Brockton, Exhibition Place, Parkdale Village",17,17,17,17,17,17
Business Reply Mail Processing Centre 969 Eastern,7,7,7,7,7,7
"Cabbagetown, St. James Town",25,25,25,25,25,25
Central Bay Street,30,30,30,30,30,30
"Chinatown, Grange Park, Kensington Market",69,69,69,69,69,69
Christie,8,8,8,8,8,8


In [18]:
tor_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

tor_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Yoga Studio,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
tor_onehot.shape

(935, 190)

In [20]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped.head(7)

,Neighborhood,Yoga Studio,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide, King, Richmond",0.0,0.0,0.00,0.00,0.00,0.00,0.052632,0.0,0.0,...,0.0,0.0,0.0,0.017544,0.0,0.000000,0.0,0.0,0.0,0.0
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.0,0.0,0.25,0.25,0.25,0.25,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,Berczy Park,0.0,0.0,0.00,0.00,0.00,0.00,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0,0.00,0.00,0.00,0.00,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.058824,0.0,0.0,0.0,0.0
4,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.00,0.00,0.00,0.00,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
5,"Cabbagetown, St. James Town",0.0,0.0,0.00,0.00,0.00,0.00,0.040000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
6,Central Bay Street,0.0,0.0,0.00,0.00,0.00,0.00,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Bar,American Restaurant,Asian Restaurant,Café,Seafood Restaurant,Thai Restaurant,Hotel,Pizza Place,Coffee Shop
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Wings Joint,Donut Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop
2,Berczy Park,Concert Hall,Liquor Store,Steakhouse,Belgian Restaurant,Beer Bar,Pub,Italian Restaurant,Wings Joint,Dumpling Restaurant,Fish Market
3,"Brockton, Exhibition Place, Parkdale Village",Café,Convenience Store,Coffee Shop,Breakfast Spot,Sandwich Place,Sporting Goods Shop,Restaurant,Caribbean Restaurant,Japanese Restaurant,Pizza Place
4,Business Reply Mail Processing Centre 969 Eastern,Park,Auto Workshop,Light Rail Station,Fast Food Restaurant,Farmers Market,Garden,Brewery,Dessert Shop,Dim Sum Restaurant,Flower Shop
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pizza Place,Café,Butcher,Jewelry Store,Beer Store,Japanese Restaurant,Italian Restaurant,Sandwich Place
6,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Deli / Bodega,Modern European Restaurant,Pizza Place,Bookstore,Spa,Thai Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Chinese Restaurant,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Noodle House,Bakery,Tea Room,Comfort Food Restaurant
8,Christie,Grocery Store,Café,Flower Shop,Gym / Fitness Center,Coffee Shop,Candy Store,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
9,Church and Wellesley,Gay Bar,Burger Joint,Japanese Restaurant,Coffee Shop,Park,Nightclub,Middle Eastern Restaurant,Piano Bar,Mexican Restaurant,Grocery Store


In [35]:
from sklearn.cluster import KMeans

toronto_data = toronto_data.drop(36)
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [38]:
tor_merged = toronto_data

tor_merged['Cluster Labels'] = kmeans.labels_ 

tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

tor_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Park,Other Great Outdoors,Playground,Trail,Diner,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,3,Greek Restaurant,Ice Cream Shop,Yoga Studio,Coffee Shop,Restaurant,Pub,Juice Bar,Bookstore,Italian Restaurant,Diner
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,3,Park,Intersection,Pet Store,Movie Theater,Pizza Place,Pub,Fast Food Restaurant,Burrito Place,Burger Joint,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Coffee Shop,Café,Italian Restaurant,New American Restaurant,Bakery,Seafood Restaurant,Fish Market,Cheese Shop,Latin American Restaurant,Chinese Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Photography Studio,Wings Joint,Dog Run,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,3,Breakfast Spot,Gym / Fitness Center,Gym,Grocery Store,Dog Run,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Boutique,Sushi Restaurant,Food Court,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,3,Café,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Seafood Restaurant,Sushi Restaurant,Toy / Game Store,Indian Restaurant,Thai Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Park,Wings Joint,Dog Run,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,3,Coffee Shop,Convenience Store,Supermarket,Athletics & Sports,Cuban Restaurant,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [40]:
address_scar = 'East Toronto'
latitude_scar = 43.676357
longitude_scar = -79.293031

In [41]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighbourhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [42]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,0,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Wings Joint,Donut Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop


In [43]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,West Toronto,1,Bar,Pool,Liquor Store,Gym / Fitness Center,Fast Food Restaurant,Pharmacy,Bank,Bakery,Music Venue,Supermarket


In [44]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,2,Sandwich Place,Café,Coffee Shop,Intersection,Noodle House,Food Truck,Bookstore,French Restaurant,Pub,Chinese Restaurant
30,Downtown Toronto,2,Grocery Store,Café,Flower Shop,Gym / Fitness Center,Coffee Shop,Candy Store,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [45]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Park,Other Great Outdoors,Playground,Trail,Diner,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant
1,East Toronto,3,Greek Restaurant,Ice Cream Shop,Yoga Studio,Coffee Shop,Restaurant,Pub,Juice Bar,Bookstore,Italian Restaurant,Diner
2,East Toronto,3,Park,Intersection,Pet Store,Movie Theater,Pizza Place,Pub,Fast Food Restaurant,Burrito Place,Burger Joint,Fish & Chips Shop
3,East Toronto,3,Coffee Shop,Café,Italian Restaurant,New American Restaurant,Bakery,Seafood Restaurant,Fish Market,Cheese Shop,Latin American Restaurant,Chinese Restaurant
4,Central Toronto,3,Photography Studio,Wings Joint,Dog Run,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant
5,Central Toronto,3,Breakfast Spot,Gym / Fitness Center,Gym,Grocery Store,Dog Run,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
6,Central Toronto,3,Boutique,Sushi Restaurant,Food Court,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
7,Central Toronto,3,Café,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Seafood Restaurant,Sushi Restaurant,Toy / Game Store,Indian Restaurant,Thai Restaurant
8,Central Toronto,3,Park,Wings Joint,Dog Run,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant
9,Central Toronto,3,Coffee Shop,Convenience Store,Supermarket,Athletics & Sports,Cuban Restaurant,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [46]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
